<a href="https://colab.research.google.com/github/mlabonne/chessllm/blob/main/Chess_LLM_Arena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title # ♟️ Chess LLM Arena
# @markdown https://gist.github.com/chessllm


import os
import time

import chess
import chess.svg
import gistyc
import outlines.models as models
import outlines.text.generate as generate
import pandas as pd
from google.colab import userdata
from IPython.display import Image as IPythonImage
from IPython.display import clear_output, update_display

from chessllm.chess_utils import (
    create_gif,
    determine_termination,
    download_file,
    generate_regex,
    init_board,
    render_board_to_image,
    save_pgn,
    save_result_file,
    write_pgn,
)
from chessllm.elo import print_elo_ratings, update_elo_ratings

os.environ["TOKENIZERS_PARALLELISM"] = "false"
GITHUB_TOKEN = userdata.get("chessllm")
RESULT_GIST_ID = "c491299e7b8a45a61ce5403a70cf8656"
LEAD_GIST_ID = "696115fe2df47fb2350fcff2663678c9"

# Initialize model
# @markdown ## Models
model_id_white = "mlabonne/chesspythia-70m"  # @param {type: "string"}
model_id_black = "BlueSunflower/Pythia-160M-chess"  # @param {type: "string"}
model_white = models.transformers(model_id_white)
model_black = models.transformers(model_id_black)

# @markdown ## Parameters
TIME_BUDGET = 120  # @param {type: "number"} # 60 = 1 minute
prompt = "1."  # @param {type: "string"}

# Initialize the chess board


board, board_images, pgn_moves, move_number, time_budget_white, time_budget_black = (
    init_board(TIME_BUDGET)
)

# Download results
url1 = f"https://gist.githubusercontent.com/chessllm/{RESULT_GIST_ID}/raw"
download_file(url1, "chessllm_results.csv")

# Load ELO ratings for each model
url2 = f"https://gist.githubusercontent.com/chessllm/{LEAD_GIST_ID}/raw"
download_file(url2, "chessllm_leaderboard.csv")
elo_ratings_df = pd.read_csv("chessllm_leaderboard.csv")
clear_output(wait=True)

# Game loop


while not board.is_game_over():
    start_time = time.time()

    # Generate move
    regex_pattern = generate_regex(board)
    current_model = model_white if board.turn == chess.WHITE else model_black
    guided = generate.regex(current_model, regex_pattern, max_tokens=10)(prompt)

    try:
        # Parse move
        move_san = guided.strip()
        move = board.parse_san(move_san)
        if move not in board.legal_moves:
            print(f"Illegal move: {move_san}")
            break
        board.push(move)

        # Write move
        if board.turn == chess.BLACK:
            move_str = f"{move_number}. {move_san} "
            move_number += 1
        else:
            move_str = f"{move_san} "
        pgn_moves += move_str

        image = render_board_to_image(board, board_images)

        update_display(image, display_id="chess_board")

        # Update time budget
        end_time = time.time()
        move_duration = end_time - start_time

        # Check the chess clock for each player
        if board.turn == chess.WHITE:
            time_budget_black -= move_duration
            if time_budget_black <= 0:
                result = "1-0"
                break
        else:
            time_budget_white -= move_duration
            if time_budget_white <= 0:
                result = "0-1"
                break

        print(f"White time left: {time_budget_white:.2f}s")
        print(f"Black time left: {time_budget_black:.2f}s")
        clear_output(wait=True)

    except ValueError:
        print(f"Invalid move: {guided}")
        break

# Get result
if result is None:
    result = board.result()

# Create PGN
termination = determine_termination(board, time_budget_white, time_budget_black)
final_pgn = write_pgn(
    pgn_moves, model_id_white, model_id_black, result, TIME_BUDGET, termination
)
file_name = f"{model_id_white.split('/')[-1]}_vs_{model_id_black.split('/')[-1]}"
pgn_id = save_pgn(final_pgn, file_name, GITHUB_TOKEN)

# Save results
save_result_file(
    pgn_id, model_id_white, model_id_black, result, GITHUB_TOKEN, RESULT_GIST_ID
)

# Create and display the GIF
clear_output(wait=True)
create_gif(board_images, file_name + ".gif", duration=400)

# Print ELO ratings

print_elo_ratings(model_id_white, model_id_black, elo_ratings_df, result)

IPythonImage(filename=file_name + ".gif")

In [ ]:
# Update ELO ratings for each model
chess_data = pd.read_csv("chessllm_results.csv")
elo_ratings = update_elo_ratings(chess_data)

# Convert the dictionary to a DataFrame for better display
elo_ratings_df = pd.DataFrame(elo_ratings.items(), columns=["Model", "ELO Rating"])

# Round the ELO ratings to the nearest integer
elo_ratings_df["ELO Rating"] = elo_ratings_df["ELO Rating"].round().astype(int)

elo_ratings_df.sort_values(by="ELO Rating", ascending=False, inplace=True)
elo_ratings_df.reset_index(drop=True, inplace=True)
elo_ratings_df.to_csv("chessllm_leaderboard.csv", index=False)

# Upload chessllm_leaderboard.csv to GIST
gist_api = gistyc.GISTyc(auth_token=GITHUB_TOKEN)
response_update_data = gist_api.update_gist(
    file_name="chessllm_leaderboard.csv", gist_id={LEAD_GIST_ID}
)

elo_ratings_df